In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
from torch.autograd import variable
import numpy as np
import gym
import random

In [2]:
env = gym.make('CartPole-v1')

In [3]:
EPISODE = 1000
EPSILON = 1
RATE = .0001
GAMMA = .95

In [4]:
class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.data = []
        self.fc1 = nn.Linear(4, 64)
        self.outlayer = nn.Linear(64, 2)
        self.optimizer = optim.Adam(self.parameters(), RATE)
        
    def pi(self, x):
        x = F.relu(self.fc1(x))
        x = F.softmax(self.outlayer(x), dim=0)
        return x
    
    def train(self):
        discounted = 0
        for r, log_p in self.data[::-1]:
            discounted = r + GAMMA * discounted
            # loss 정의
            loss = - log_p * discounted
            # 그래디언트 계산
            loss.backward()
            self.optimizer.zero_grad()
            self.optimizer.step()
        self.data = []

In [6]:
state = env.reset()
ep = 1
net = Policy()

while(ep < EPISODE):
    done = False
    total_reward = 0
    
    while(not done):
        #state = np.reshape(state, [1, 4])
        env.render()

        # policy run
        action_prob = net.pi(torch.from_numpy(state).float())
        m = Categorical(action_prob)
        a = m.sample()

        # action 선택
        if(EPSILON < random.randrange(0,1)):
            action = env.action_space.sample()
        else:
            action = a.item()

        # step
        state_next, reward, done, _ = env.step(action)

        # reward 추가
        total_reward += reward
        
        if(done):
            reward = -100

        # data 추가
        net.data.append((reward, torch.log(action_prob[a])))

        # state 갱신
        state = state_next

        # end episode
        if(done):
            print(total_reward)
            total_reward = 0
            ep += 1
            
            # env 초기화
            state = env.reset()
            EPSILON = 1 / (ep / 100 + 1)
            
            # 학습
            net.train()
env.close()

18.0
15.0
24.0
19.0
12.0
14.0
17.0
15.0
17.0
19.0
40.0
23.0
9.0
28.0
23.0
14.0
15.0
14.0
19.0
16.0
17.0
41.0
28.0
23.0
34.0
25.0
23.0
26.0
27.0
14.0
37.0
11.0
17.0
12.0
28.0
45.0
17.0
35.0
21.0
25.0
25.0
11.0
20.0
14.0
20.0
15.0
20.0
30.0
25.0
44.0
30.0
20.0
20.0
18.0
25.0
25.0
30.0
16.0
11.0
27.0
39.0
16.0
26.0
14.0
32.0
21.0
23.0
16.0
17.0
40.0
14.0
57.0
22.0
22.0
43.0
44.0
16.0
24.0
21.0
22.0
13.0
22.0
36.0
18.0
14.0
10.0
21.0
25.0
24.0
24.0
28.0
20.0
42.0
12.0
32.0
11.0
20.0
10.0
17.0
10.0
24.0
34.0
13.0
44.0
17.0
41.0
37.0
9.0
12.0
18.0
44.0
46.0
16.0
26.0
27.0
15.0
17.0
13.0
12.0
33.0
14.0
13.0
56.0
13.0
11.0
15.0
17.0
17.0
31.0
41.0
15.0
46.0
12.0
54.0
14.0
25.0
14.0
25.0
22.0
41.0
13.0
18.0
16.0
9.0
36.0
12.0
27.0
53.0
51.0
16.0
14.0
35.0
35.0
27.0
33.0
23.0
27.0
16.0
19.0
10.0
41.0
39.0
12.0
64.0
27.0
27.0
20.0
22.0
24.0
12.0
18.0
13.0
31.0
19.0
19.0
18.0
19.0
11.0
11.0
23.0
22.0
13.0
32.0
42.0
13.0
18.0
19.0
13.0
15.0
40.0
17.0
12.0
22.0
14.0
46.0
31.0
18.0
18.0
31.0
33.0
18.

KeyboardInterrupt: 